In [54]:
#import libraries
from time import time, sleep
import random
import urllib
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
import bs4
from bs4 import BeautifulSoup
import re
from IPython.display import clear_output
import os
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import visibility_of_element_located
from selenium.webdriver.common.action_chains import ActionChains
import winsound

In [55]:
url_template = "https://www.pets4homes.co.uk/{}/{}/local/{}/"
url_template_pages = "https://www.pets4homes.co.uk/{}/{}/local/{}/page-{}/"
listing_type = ['sale','adoption','stud']
pet_type = ['dogs','cats','reptiles','rodents','rabbits','horses','invertebrates','livestock', 'poultry', 'birds','fish']
cities = ['aberdeen','ayr_st-ives', 'barnsley_wimborne','basingstoke','bath','bedfordshire','birmingham','blackpool_plymouth',
         'bolton_appleby-in-westmorland', 'bournemouth', 'bradford_holsworthy', 'bridgend_brechin', 'brighton',
          'bristol_south-west-england', 'cambridge_gloucester', 'cardiff-county', 'carlisle', 'chelmsford', 'cheltenham',
         'chester', 'chesterfield', 'colchester', 'cornwall', 'coventry', 'crawley_witney', 'crewe', 'darlington', 'derby', 
         'devon', 'doncaster', 'dorset', 'dudley_cramlington', 'dundee', 'durham', 'eastbourne', 'edinburgh', 'essex', 'exeter',
         'glasgow', 'gloucester', 'grimsby', 'guildford', 'hampshire', 'hereford', 'hertfordshire', 'huddersfield','hull',
          'ipswich', 'kent', 'lancashire','leeds_maidstone','leicester','lincoln', 'liverpool', 'london','luton_newton-abbot',
         'maidstone', 'manchester', 'merseyside', 'middlesbrough', 'milton-keynes', 'newcastle-upon-tyne', 
          'newport_saffron-walden', 'northampton', 'norwich', 'nottingham', 'oxford', 'peterborough', 'plymouth', 
          'poole_nantwich']

In [56]:
def check_captcha (soup):
    if 'I am human' in soup.get_text():
            print('captcha detected')
            duration = 5000  # milliseconds
            freq = 440  # Hz
            winsound.Beep(freq, duration) #sound alert, might get irritating, probably won't work on mac
            input("Press Enter to continue...")#this would pause the code until i press enter
    else:
        print('no captcha detected, proceed to scrape')

In [57]:
def scrape_header(soup):
    df = pd.DataFrame(columns=["Title","Description","photo_link","category", "price", "url",
                                              "seller_type", "seller_name", 'listing_type','pet_type'])
    for each in soup.find_all("script", type="application/ld+json"):
        entries = each.get_text()
        split = entries.split("\n")
        Title = split[3].replace('"name": "', '').replace('",', '')
        Description = split[4].replace('"description": "', '').replace('",', '').replace('\n,', '')
        Photo = split[5].replace('"image": "', '').replace('",', '')
        Category = split[7].replace('"category": "', '').replace('",', '')
        Price = split[11].replace('"price": "', '').replace('",', '')
        url =split[13].replace('"url": "', '').replace('",', '')
        seller_type = split[15].replace('"type": "', '').replace('",', '')
        seller_name = split[16].replace('"name":  "', '').replace('"', '')
        df = df.append({"Title":Title,"Description":Description,"photo_link":Photo,"category":Category, 
                                      "price":Price,"url":url, "seller_type":seller_type, "seller_name":seller_name,
                                      "listing_type":listing, "pet_type": pets},ignore_index=True)
    return df

In [58]:
def scrape_content(soup):
    df1 = pd.DataFrame(columns=["Title","price","species", "age", "gender", "description",
                                "seller_name", "seller_location", "seller_type", "listing_type","pet_type"])
    for each in soup.find_all("div", class_="rn lj"):
        entries = each.get_text()
        title = each.find('h2', {'class':'Bn'}).text
        price = each.find('span', {'class':'Cn'}).text
        info = each.find_all('span', {'class':'qv'})
        species = info[0].text
        try:
            age = info[1].text.replace('Age:  ', '')
        except:
            age = 'unknown'
        try:
            gender = info[2].text
        except:
            gender = 'unknown'
        description = each.find('span', {'class':'Dn'}).text
        seller_name = each.find('span', {'class':'Jn'}).text
        seller_location = each.find('span', {'class':'gt'}).text
        seller_type = each.find('div', {'class':'ef Ql ff'}).text
        df1 = df1.append({"Title":title, "price":price, "species":species, "age":age, "gender":gender,
                              "description":description, "seller_name":seller_name, "seller_location": seller_location,
                              "seller_type":seller_type, "listing_type":listing, "pet_type": pets},ignore_index=True)
    return df1

In [ ]:
for city in set (cities):
    for pets in set(pet_type):
        for listing in set(listing_type):
            page = 1
            while page > 0:
                clear_output(wait=True)
                file = 'data_by_cities/header_'+str(city)+'.csv'
                file1 = 'data_by_cities/body_'+str(city)+'.csv'
                try: 
                    df_header = pd.read_csv(file , sep='\t', encoding='utf-8', index_col=0)
                    print('file successfully opened')
                except:
                    print('file not found, creating new file')
                    df_header = pd.DataFrame(columns=["Title","Description","photo_link","category", "price", "url",
                                              "seller_type", "seller_name", 'listing_type','pet_type'])
                    df_header.to_csv(file, sep='\t', encoding='utf-8')
                try: 
                    df_body = pd.read_csv(file1 , sep='\t', encoding='utf-8', index_col=0)
                    print('file successfully opened')
                except:
                    print('file not found, creating new file')
                    df_body = pd.DataFrame(columns=["Title","price","species", "age", "gender", "description",
                                                    "seller_name", "seller_location", "seller_type" , 
                                                    "listing_type","pet_type"])
                    df_body.to_csv(file1, sep='\t', encoding='utf-8')
                headersize = df_header.shape
                bodysize = df_body.shape
                if page == 1:
                    url = url_template.format(listing, pets, city)
                else:
                    url = url_template_pages.format(listing, pets, city, page)
                options = webdriver.ChromeOptions()
                browser = webdriver.Chrome(ChromeDriverManager().install(),options = options)
                browser.get(url)
                title = (
                    WebDriverWait(driver=browser, timeout=15)
                    .until(visibility_of_element_located((By.CSS_SELECTOR, "h1"))).text
                )
                content = browser.page_source
                print(city,pets,listing,page)
                soup = BeautifulSoup(content, 'lxml')
                check_captcha(soup)
                if 'We found 0' in soup.get_text():
                    print('no result found')
                    break
                df_header = df_header.append(scrape_header(soup))
                df_body = df_body.append(scrape_content(soup))
                #drop duplicates
                df_header = df_header[~df_header.duplicated(subset=['Title','Description', 'photo_link','category','url','seller_type','listing_type','pet_type'])].reset_index(drop=True)
                df_header.to_csv(file, sep='\t', encoding='utf-8')
                df_body = df_body[~df_body.duplicated()].reset_index(drop=True)
                df_body.to_csv(file1, sep='\t', encoding='utf-8')
                if df_body.shape == bodysize:
                    if df_header.shape == headersize:
                        print('no new data')
                    else:
                        winsound.Beep(600, 1000)
                        input('html tags is likely changed, please kill program')
                next_page = soup.find('a', {'class':'Fb gh Jb'})
                if next_page == None:
                    print('next page not found')
                    break
                else:
                    print ('next page found')
                    print('going to next page')
                    sleep(random.randint(10,15))
                    page +=1
            print('going to next listing type')
            sleep(random.randint(10,15))
        print('going to next pet type')
        sleep(random.randint(10,15))
    print('going to next city')

file successfully opened
file successfully opened



[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\silve\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


london poultry adoption 1
no captcha detected, proceed to scrape
next page not found
going to next listing type


In [11]:
headersize

(66, 10)

In [12]:
bodysize

(66, 11)

In [13]:
df_body.shape

(66, 11)

In [14]:
df_header.shape

(70, 10)

In [48]:
df_header.head()

,Title,Description,photo_link,category,price,url,seller_type,seller_name,listing_type,pet_type
0,unsexed green tree python + 60cm vivtek,~3years old gtp biak x sorong\n\nTypical...,https://assets.pets4homes.co.uk/image/16...,Pets > Reptiles > Python Snake,1000,/classifieds/z9ld6e8uk-unsexed-green-t...,Person,Patryk M.,sale,reptiles
1,Leopard gecko (f),Selling this leopard gecko I've had the ...,https://assets.pets4homes.co.uk/image/97...,Pets > Reptiles > Gecko,80,/classifieds/x19l6ke04-leopard-gecko-f...,Person,Dana I.,sale,reptiles
2,rexy,rexy needs a new home ASP\n sadly I don'...,https://assets.pets4homes.co.uk/image/54...,Pets > Reptiles > Iguana,100,/classifieds/sxvfb08xd-rexy-aberdeen/,Person,Gillian W.,sale,reptiles
3,Panther Chameleon and full set up,Blue is a 6 month old Nosey Mitsio chame...,https://assets.pets4homes.co.uk/image/81...,Pets > Reptiles > Chameleon,550,/classifieds/8omv9mhc0-panther-chamele...,Person,Craig H.,sale,reptiles
4,Hand reared white Indian ringnecks,I have 2 stunning white Indian ringnecks...,https://assets.pets4homes.co.uk/image/3b...,Pets > Birds > Parakeets,300,/classifieds/l2t7nnf71-hand-reared-whi...,Person,Dave T.,sale,birds


In [49]:
df_header.tail()

,Title,Description,photo_link,category,price,url,seller_type,seller_name,listing_type,pet_type
179,Mini lop,Male mini lop 8 months old \nIs a little...,https://assets.pets4homes.co.uk/image/c6...,Pets > Rabbits > Mini Lop,60,/classifieds/qajwajci4-mini-lop-ellon/,Organization,Miss S.,sale,rabbits
180,Mini dwarf lop doe,Mini dwarf lop doe\nHarlequin colouring\...,https://assets.pets4homes.co.uk/image/91...,Pets > Rabbits > Mini Lop,50,/classifieds/wqozddeer-mini-dwarf-lop-...,Organization,Samantha L.,sale,rabbits
181,BEAUTIFUL Boys Bunnies REX & LOP💓 rare ...,***Only for loving homes***\nFor sale 3 ...,https://assets.pets4homes.co.uk/image/23...,Pets > Rabbits > Mixed Breed,50,/classifieds/yqaoi6nmb-beautiful-bunni...,Person,Violetta O.,sale,rabbits
182,Pet mice,"Fancy mice for sale, some curly coated a...",https://assets.pets4homes.co.uk/image/7b...,Pets > Rodents > Mouse,5,/classifieds/odnrdwop0-pet-mice-aberdeen/,Person,Emma J.,sale,rodents
183,2x giant african snails,Comes with everything I have for them fo...,https://assets.pets4homes.co.uk/image/ac...,Pets > Invertebrates > Snails,20,/classifieds/mspuowzkx-2x-giant-africa...,Person,Danielle S.,sale,invertebrates


In [ ]:
df_header = df_header[~df_header.duplicated()].reset_index(drop=True)

In [47]:
df_header.duplicated(subset=['Title','Description', 'photo_link','category','url','seller_type','listing_type','pet_type'])

0      False
1      False
2      False
3      False
4      False
       ...  
179    False
180    False
181    False
182    False
183    False
Length: 184, dtype: bool

In [50]:
df_body.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Title            184 non-null    object
 1   price            184 non-null    object
 2   species          184 non-null    object
 3   age              184 non-null    object
 4   gender           184 non-null    object
 5   description      184 non-null    object
 6   seller_name      184 non-null    object
 7   seller_location  184 non-null    object
 8   seller_type      184 non-null    object
 9   listing_type     184 non-null    object
 10  pet_type         184 non-null    object
dtypes: object(11)
memory usage: 15.9+ KB
